## RAG con early chunking y Qdrant intentando adjuntar metadata para tener mayor contexto

In [ ]:
import os
import json
import nltk
from dotenv import load_dotenv
import openai
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import numpy as np
import uuid

# Cliente de Qdrant y modelos
from qdrant_client import QdrantClient
from qdrant_client.http import models

# ------------------------------
# 1. CONFIGURACIÓN INICIAL
# ------------------------------

# Descargar recursos necesarios de NLTK
nltk.download('punkt')

# Cargar variables de entorno (.env debe contener OPENAI_API_KEY, QDRANT_URL, QDRANT_API_KEY, etc.)
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

# Cargar el modelo de embeddings (por ejemplo, all-MiniLM-L6-v2)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_dim = 384  # Dimensión del embedding para este modelo

# Parámetros para el chunking
max_chunk_tokens = 600
overlap_tokens = 80

# Archivo local para guardar la información de PDFs indexados
INDEXED_FILES_PATH = "indexed_files.json"

# ------------------------------
# 2. CONEXIÓN A QDRANT CLOUD
# ------------------------------

qdrant_url = os.getenv("QDRANT_URL")         
qdrant_api_key = os.getenv("QDRANT_API_KEY")
qdrant = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)
collection_name = "pdf_chunks"

# Crear la colección en Qdrant si no existe
try:
    qdrant.get_collection(collection_name=collection_name)
    print('La colección de Qdrant ya existe.')
except Exception as e:
    print("Creando colección en Qdrant Cloud...")
    qdrant.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(size=embedding_dim, distance=models.Distance.COSINE)
    )

# ------------------------------
# 3. FUNCIÓN DE CHUNKING (EARLY CHUNKING)
# ------------------------------

def late_chunking(text, max_chunk_tokens=max_chunk_tokens, overlap_tokens=overlap_tokens):
    """
    Fragmenta el texto en chunks de hasta 'max_chunk_tokens' tokens,
    agregando un solapamiento de 'overlap_tokens' entre fragmentos.
    Se utiliza el tokenizador de oraciones de NLTK para evitar cortar oraciones.
    """
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = ""
    current_tokens = 0

    for sentence in sentences:
        sentence_tokens = sentence.split()
        num_tokens = len(sentence_tokens)
        # Si se supera el límite al agregar la oración, se guarda el chunk actual
        if current_tokens + num_tokens > max_chunk_tokens:
            chunks.append(current_chunk.strip())
            # Mantener solapamiento: conservar los últimos 'overlap_tokens' tokens
            current_chunk_tokens = current_chunk.split()
            current_chunk = " ".join(current_chunk_tokens[-overlap_tokens:]) if overlap_tokens > 0 else ""
            current_tokens = len(current_chunk.split())
        current_chunk += " " + sentence
        current_tokens += num_tokens

    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# ------------------------------
# 4. CARGAR Y PROCESAR PDFs CON EARLY CHUNKING (Solo nuevos/modificados)
# ------------------------------

def load_indexed_files(cache_path):
    """Carga el diccionario de PDFs ya indexados desde un archivo JSON."""
    if os.path.exists(cache_path):
        try:
            with open(cache_path, "r") as f:
                return json.load(f)
        except Exception as e:
            print(f"Error al cargar el archivo de cache: {e}")
    return {}

def save_indexed_files(cache_path, indexed_files):
    """Guarda el diccionario de PDFs indexados en un archivo JSON."""
    with open(cache_path, "w") as f:
        json.dump(indexed_files, f)

def load_and_chunk_pdfs(root_folder, indexed_files):
    """
    Recorre recursivamente 'root_folder' para extraer el texto de cada PDF.
    Si el PDF ya fue procesado (según su ruta y marca de tiempo), se omite.
    Divide el contenido en chunks y añade metadata (bloque, nombre del archivo, ruta y mod_time).
    
    Retorna:
      - Una lista de diccionarios con keys: 'text' y 'metadata'.
    """
    chunks = []
    
    for subdir, dirs, files in os.walk(root_folder):
        for file in files:
            if file.lower().endswith('.pdf'):
                file_path = os.path.join(subdir, file)
                mod_time = os.path.getmtime(file_path)
                # Verificar si el archivo ya fue indexado y no se ha modificado
                if file_path in indexed_files and indexed_files[file_path] == mod_time:
                    continue  # Se salta este archivo, ya fue indexado
                block_name = os.path.basename(subdir)
                try:
                    reader = PdfReader(file_path)
                    full_text = ""
                    for page in reader.pages:
                        page_text = page.extract_text()
                        if page_text:
                            full_text += page_text + "\n"
                    # Aplicar chunking al texto completo
                    pdf_chunks = late_chunking(full_text)
                    for chunk in pdf_chunks:
                        # Incluir metadata en cada chunk para proporcionar contexto al LLM
                        chunk_text = f"[Bloque: {block_name} | PDF: {file}]\n{chunk}"
                        chunks.append({
                            "text": chunk_text,
                            "metadata": {
                                "block": block_name,
                                "file": file,
                                "path": file_path,
                                "mod_time": mod_time
                            }
                        })
                    # Actualizar el diccionario de archivos indexados
                    indexed_files[file_path] = mod_time
                except Exception as e:
                    print(f"Error al leer {file_path}: {e}")
    return chunks

# Ruta de la carpeta que contiene los PDFs (ajusta según tus necesidades)
pdf_root = "temario_opos"

# Cargar la cache de PDFs indexados
indexed_files = load_indexed_files(INDEXED_FILES_PATH)

# Procesar solo PDFs nuevos o modificados
new_chunks = load_and_chunk_pdfs(pdf_root, indexed_files)
if new_chunks:
    print(f"Se han generado {len(new_chunks)} nuevos chunks desde '{pdf_root}'.")
else:
    print("No se encontraron PDFs nuevos o modificados. Se utilizará el índice existente en Qdrant.")

# ------------------------------
# 5. INDEXACIÓN DE CHUNKS NUEVOS EN QDRANT CLOUD (si hay)
# ------------------------------

if new_chunks:
    points = []
    for chunk in new_chunks:
        # Calcular embedding del chunk
        embedding = embedding_model.encode(chunk["text"], convert_to_numpy=True)
        # Crear un punto para Qdrant (usando un id único)
        point = models.PointStruct(
            id=str(uuid.uuid4()),
            vector=embedding.tolist(),
            payload={
                "text": chunk["text"],
                "block": chunk["metadata"]["block"],
                "file": chunk["metadata"]["file"],
                "path": chunk["metadata"]["path"],
                "mod_time": chunk["metadata"]["mod_time"]
            }
        )
        points.append(point)
    
    print("Indexando nuevos chunks en Qdrant Cloud...")
    qdrant.upsert(collection_name=collection_name, points=points)
    print("Indexación de nuevos chunks completada.")
    # Actualizar la cache local de archivos indexados
    save_indexed_files(INDEXED_FILES_PATH, indexed_files)

# ------------------------------
# 6. FUNCIÓN PARA RESPONDER CONSULTAS (RAG)
# ------------------------------

def answer_query(query, top_k=5, token_budget=2000):
    """
    Procesa la consulta del usuario:
      - Genera el embedding de la consulta.
      - Realiza una búsqueda en Qdrant Cloud para recuperar los chunks más relevantes.
      - Combina los chunks hasta alcanzar un presupuesto de tokens y forma el contexto.
      - Consulta la API de OpenAI para generar la respuesta basada en el contexto.
    """
    # Generar embedding para la consulta
    query_embedding = embedding_model.encode(query, convert_to_numpy=True)
    
    # Buscar los chunks más relevantes en Qdrant Cloud
    search_result = qdrant.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=top_k,
        with_payload=True
    )
    
    # Seleccionar y combinar chunks según el token_budget
    selected_chunks = []
    total_tokens = 0
    for res in search_result:
        chunk_text = res.payload.get('text', '')
        tokens = len(chunk_text.split())
        if total_tokens + tokens <= token_budget:
            selected_chunks.append(chunk_text)
            total_tokens += tokens
        else:
            break
    
    context = "\n\n".join(selected_chunks)
    prompt = f"Contexto:\n{context}\n\nPregunta: {query}"
    
    # Llamar a la API de ChatCompletion
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Eres un asistente educativo experto. Responde solo en base al contexto que te proporciono."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0,
    )
    
    return response.choices[0].message.content.strip()

# ------------------------------
# 7. BUCLE DE INTERACCIÓN CONTINUA
# ------------------------------

if __name__ == "__main__":
    print("Bienvenido al chatbot. Escribe 'salir' o 'exit' para terminar la conversación.")
    while True:
        consulta = input("Introduce tu consulta: ")
        if consulta.lower() in ["salir", "exit"]:
            print("Finalizando la conversación. ¡Hasta luego!")
            break
        respuesta = answer_query(consulta)
        print("\nRespuesta del chatbot:")
        print(respuesta)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joseluis.fernandez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
